# Expression remapper

This notebook is a simple app that allows users to map different facial expressions and motions to keyboard inputs. I aim to use this to play a video game like Dark Souls as a test.

Using MediaPipe, and following their [face landmark detection guide](https://developers.google.com/mediapipe/solutions/vision/face_landmarker/python).

## Install dependencies

#### Mediapipe

In [ ]:
!pip install mediapipe
# opencv as well

## Import dependencies

In [1]:
# Mediapipe for face landmark detection
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# OpenCV for drawing utilities and webcam input
import cv2

## Load the model

In [2]:
model_path = 'models/face_landmarker.task'

## Create the Face Landmarker task

In [20]:
import mediapipe as mp
import math

BaseOptions = mp.tasks.BaseOptions
FaceLandmarker = mp.tasks.vision.FaceLandmarker
FaceLandmarkerOptions = mp.tasks.vision.FaceLandmarkerOptions
FaceLandmarkerResult = mp.tasks.vision.FaceLandmarkerResult
VisionRunningMode = mp.tasks.vision.RunningMode

# Real yaw pitch and roll
yaw = 0.0 # Rotate left/right
pitch = 0.0 # Tilt up/down
roll = 0.0 # Tilt left/right
# Calibrated yaw pitch and roll values
calib_yaw = 0
calib_pitch = 0
calib_roll = 0
# Base values used to calibrate what the 'rest' positions/values for yaw/pitch/roll should be.
rest_yaw = 0.0
rest_pitch = 0.0
rest_roll = 0.0

def calibrate_yaw_pitch_roll():
    # Get access to global rest pos variables (I don't like python..)
    # then set them to the current yaw, pitch, and roll of the user's head.
    global yaw
    global pitch
    global roll
    global rest_yaw
    global rest_pitch
    global rest_roll
    rest_yaw = yaw
    rest_pitch = pitch
    rest_roll = roll
    
    # Print results
    print("resting yaw is: " + str(rest_yaw))
    print("resting pitch is: " + str(rest_pitch))
    print("resting roll is: " + str(rest_roll))
    

# Create a face landmarker instance with the live stream mode:
def print_result(result: FaceLandmarkerResult, output_image: mp.Image, timestamp_ms: int):
    #print('face landmarker result: {}'.format(result))
    #print('facial transformation matrixes: {}'.format(result.facial_transformation_matrixes))
    
    # Below code for calculating the orientation of the head has been adapted from Nico Nielsen's code
    # https://github.com/niconielsen32/ComputerVision/blob/master/headPoseEstimation.py
    # If any landmarks have been detected..
    if result.face_landmarks:
        mtrix = result.facial_transformation_matrixes[0]
        global yaw
        global pitch
        global roll
                
        # Get the yaw pitch and roll from the transformation matrix (yuck.. I still barely understand these darn things :/)
        if mtrix[0][0] == 1.0:
            yaw = math.atan2(mtrix[0][2], mtrix[2][3])
            pitch = 0
            roll = 0
        
        elif mtrix[0][0] == -1.0:
            yaw = math.atan2(mtrix[0][0], mtrix[2][3])
            pitch = 0
            roll = 0
        
        else:
            yaw = math.atan2(-mtrix[2][0], mtrix[0][0])
            pitch = math.atan2(-mtrix[1][2], mtrix[1][1])
            roll = math.asin(mtrix[1][0])
        
        # Calculate the real usable CALIBRATED yaw pitch roll values
        global calib_yaw
        global calib_pitch
        global calib_roll
        calib_yaw = yaw - rest_yaw
        calib_pitch = pitch - rest_pitch
        calib_roll = roll - rest_roll
        
        # CALIBRATE THESE VALUES TO COMPARE BY ASKING USER TO LOOK DIFFERENT WAYS
        # Yaw (look left/right)
        print("yaw is: {}".format(math.degrees(calib_yaw)))
        if calib_yaw > 0:
            print("looking left")
        elif calib_yaw < 0:
            print("looking right")
        
#         # Pitch (look up/down)
#         print("pitch is: {}".format(math.degrees(calib_pitch)))
#         if calib_pitch > 0:
#             print("looking down")
#         elif calib_pitch < 0:
#             print("looking up")
        
#         # Roll (rilt left/right)
#         print("roll is: {}".format(math.degrees(calib_roll)))
#         if calib_roll > 0:
#             print("looking rolled right")
#         elif calib_roll < 0:
#             print("looking rolled left")
                
    

options = FaceLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.LIVE_STREAM,
    result_callback=print_result,
    output_facial_transformation_matrixes=True, ## TRY CHANGING THIS
    output_face_blendshapes=False) ## TRY CHANGING THIS

with FaceLandmarker.create_from_options(options) as landmarker:
    # The landmarker is initialized. Use it here.
    # Use OpenCV’s VideoCapture to start capturing from the webcam.
    video = cv2.VideoCapture(0)
    
    # Initialise the frame timestamp (MAKE THIS IN ms?)
    frame_timestamp = 0
    
    # Tell the user to calibrate
    print("---- Calibration -----")
    print("Please look straight ahead at your screen, then press ENTER when comfortable")
    
    # Create a loop to read the latest frame from the camera using VideoCapture#read()
    while video.isOpened():
        ret, frame = video.read()
        
        # Convert the frame received from OpenCV to a MediaPipe Image object.
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
        
        # Send live image data to perform face landmarking.
        # The results are accessible via the `result_callback` provided in
        # the `PoseLandmarkerOptions` object.
        # The pose landmarker must be created with the live stream mode.
        landmarker.detect_async(mp_image, frame_timestamp)
        
        # Increment the timestamp
        frame_timestamp += 1
        
        # Draw the output
        cv2.imshow('window', frame)
        
        # Listen for keyboard input
        keyPressed = cv2.waitKey(5)
        if keyPressed == ord('q'): # 'q': Break out of loop and exit program
            break
        elif keyPressed == 13: # 'Enter': Calibrate yaw+pitch+roll
            calibrate_yaw_pitch_roll()

# Clean up
video.release()
cv2.destroyAllWindows()

---- Calibration -----
Please look straight ahead at your screen, then press ENTER when comfortable
yaw is: -3.971651664921558
looking right
yaw is: -4.090741771656154
looking right
yaw is: -4.137358807937844
looking right
yaw is: -4.156991370399107
looking right
yaw is: -4.269128828365366
looking right
yaw is: -4.7214098759538325
looking right
yaw is: -5.124227986846363
looking right
yaw is: -5.390877025285213
looking right
yaw is: -5.554832010928515
looking right
yaw is: -5.92529044407763
looking right
yaw is: -6.3145466221712505
looking right
yaw is: -6.524491335173074
looking right
yaw is: -6.820056178017934
looking right
yaw is: -7.503640668885627
looking right
yaw is: -8.097947529507357
looking right
yaw is: -8.80693265593819
looking right
yaw is: -9.602358441852349
looking right
yaw is: -10.177591735172845
looking right
yaw is: -10.540246009132453
looking right
yaw is: -11.212786554584463
looking right
yaw is: -12.140951838308395
looking right
yaw is: -12.732708554303153
looking

yaw is: -16.703653429632986
looking right
yaw is: -16.67533988083129
looking right
yaw is: -16.66124556939936
looking right
yaw is: -16.68734105296723
looking right
yaw is: -16.70950959294172
looking right
yaw is: -16.709189205285043
looking right
yaw is: -16.706965242761676
looking right
yaw is: -16.728954166135058
looking right
yaw is: -16.713982855551112
looking right
yaw is: -16.71819611468456
looking right
yaw is: -16.71703058585973
looking right
yaw is: -16.6873889817552
looking right
yaw is: -16.634200515993573
looking right
yaw is: -16.596518317198782
looking right
yaw is: -16.55869317361466
looking right
yaw is: -16.546514097504556
looking right
yaw is: -16.551132246501723
looking right
yaw is: -16.565248329784865
looking right
yaw is: -16.578017643210863
looking right
yaw is: -16.47131626727623
looking right
yaw is: -16.422056584480764
looking right
yaw is: -16.4035664894472
looking right
yaw is: -16.21583756157798
looking right
yaw is: -16.012126044081896
looking right
yaw i

yaw is: 22.803153237580652
looking left
yaw is: 18.983796140353217
looking left
yaw is: 14.104511834706281
looking left
yaw is: 12.092480070097942
looking left
yaw is: 10.276757480587353
looking left
yaw is: 9.073093521427502
looking left
yaw is: 8.103290801381755
looking left
yaw is: 7.291133213098235
looking left
yaw is: 6.505497870193036
looking left
yaw is: 5.890914659063595
looking left
yaw is: 5.337367674977512
looking left
yaw is: 4.810195932845677
looking left
yaw is: 4.3405587288059495
looking left
yaw is: 3.87358757228926
looking left
yaw is: 3.4906199578197152
looking left
yaw is: 3.1371824840050273
looking left
yaw is: 2.828627695652012
looking left
yaw is: 2.5596380543930177
looking left
yaw is: 2.310828224032987
looking left
yaw is: 2.0604552644255265
looking left
yaw is: 1.8015258126226916
looking left
yaw is: 1.5523559481710036
looking left
yaw is: 1.3180689180118792
looking left
yaw is: 1.0975909629450904
looking left
yaw is: 0.8869072691183781
looking left
yaw is: 0.6

yaw is: -0.573777957022779
looking right
yaw is: -0.5826258918741914
looking right
yaw is: -0.581776403018505
looking right
yaw is: -0.5085380376354733
looking right
yaw is: -0.4472160016635961
looking right
yaw is: -0.3918016422571526
looking right
yaw is: -0.0864723270313061
looking right
yaw is: 0.0017278305239790957
looking left
yaw is: -0.009611438199242565
looking right
yaw is: -0.01765554292399938
looking right
yaw is: -0.03434433919758769
looking right
yaw is: -0.16013665705736316
looking right
yaw is: -0.2719836268907371
looking right
yaw is: -0.2971810923734336
looking right
yaw is: -0.29343391637941163
looking right
yaw is: -0.2447012639386785
looking right
yaw is: -0.18142175577897235
looking right
yaw is: -0.18481028019155046
looking right
yaw is: -0.17356412073440075
looking right
yaw is: -0.328655027814407
looking right
yaw is: -0.47544461224206513
looking right
yaw is: -0.7574319158104079
looking right
yaw is: -1.184711231818556
looking right
yaw is: -1.5165418417979804

In [13]:
-34 + (-10 * -1)

-24

In [14]:
-34 - -10

-24

In [15]:
-34 - 10

-44

In [16]:
-34 + (10 * -1)

-44